In [3]:
import pandas as pd
pd.set_option('display.max_columns', 500)

In [11]:
#listings
df_listings = pd.read_csv('data/listings.csv')

df_listings.head(4)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.23,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,43,2019-01-02,0.38,2,365
2,3330,++ Brooklyn Penthouse Guestroom ++,4177,Jbee,Brooklyn,Williamsburg,40.70856,-73.94236,Private room,70,5,39,2018-12-07,0.35,3,217
3,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365


In [12]:
df_subway = pd.read_csv('data/NYC_Transit_Subway_Entrance_And_Exit_Data.csv')
df_subway.head(4)

,Division,Line,Station Name,Station Latitude,Station Longitude,Route1,Route2,Route3,Route4,Route5,Route6,Route7,Route8,Route9,Route10,Route11,Entrance Type,Entry,Exit Only,Vending,Staffing,Staff Hours,ADA,ADA Notes,Free Crossover,North South Street,East West Street,Corner,Entrance Latitude,Entrance Longitude,Station Location,Entrance Location
0,BMT,4 Avenue,25th St,40.660397,-73.998091,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stair,YES,NaN,YES,FULL,NaN,False,NaN,False,4th Ave,25th St,SE,40.660323,-73.997952,"(40.660397, -73.998091)","(40.660323, -73.997952)"
1,BMT,4 Avenue,25th St,40.660397,-73.998091,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stair,YES,NaN,YES,NONE,NaN,False,NaN,False,4th Ave,25th St,SW,40.660489,-73.998220,"(40.660397, -73.998091)","(40.660489, -73.998220)"
2,BMT,4 Avenue,36th St,40.655144,-74.003549,N,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stair,YES,NaN,YES,FULL,NaN,False,NaN,True,4th Ave,36th St,NW,40.654490,-74.004499,"(40.655144, -74.003549)","(40.654490, -74.004499)"
3,BMT,4 Avenue,36th St,40.655144,-74.003549,N,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stair,YES,NaN,YES,FULL,NaN,False,NaN,True,4th Ave,36th St,NE,40.654365,-74.004113,"(40.655144, -74.003549)","(40.654365, -74.004113)"


In [87]:
df2 = df_subway[["Station Name","Station Latitude", "Station Longitude", "Entrance Location" ]]
df2.head(2)

,Station Name,Station Latitude,Station Longitude,Entrance Location
0,25th St,40.660397,-73.998091,"(40.660323, -73.997952)"
1,25th St,40.660397,-73.998091,"(40.660489, -73.998220)"


In [6]:
import math

def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d

def findsubwaydistance(latlong):
    return findnearestsubway(latlong, df2)

def findnearestsubway(latlong, subwaydef):
    
    shortdist=-1
    shortindex=-1
    for index, row in subwaydef.iterrows():
        dest = (row["Station Latitude"], row["Station Longitude"])
       
        nextdist = distance(latlong, dest)
        nextindex = index
        if shortdist == -1 :
            shortdist =  nextdist
            shortindex = index
        else:
            if shortdist > nextdist:
                shortdist, shortindex = nextdist, nextindex 
            
    return shortdist 

def findtransitscore(dist):
    if dist < 0.5:
        return 5
    if dist < 0.9:
        return 4
    if dist < 1.5 :
        return 3
    if dist < 2:
        return 2
    if dist < 5:
        return 1
    if dist > 5:
        return 0

In [90]:
lat1=40.64749
long1=-73.97237

lat2 = 40.660323
long2=-73.997952

print( distance((lat1, long1), (lat2, long2)) )


2.587180281603053


In [91]:
df1 = df_listings[["id", "name", "latitude", "longitude"]]
df1.head(2)

,id,name,latitude,longitude
0,2539,Clean & quiet apt home by the park,40.64749,-73.97237
1,2595,Skylit Midtown Castle,40.75362,-73.98377


In [7]:
df = pd.read_csv('data/listings_subwayrank.csv')
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,subway
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.23,6,365,0.465364
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,43,2019-01-02,0.38,2,365,0.094923
2,3330,++ Brooklyn Penthouse Guestroom ++,4177,Jbee,Brooklyn,Williamsburg,40.70856,-73.94236,Private room,70,5,39,2018-12-07,0.35,3,217,0.230424
3,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365,0.333712
4,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,239,2019-03-03,4.43,1,164,0.415913


In [8]:
df["transitscore"] = df["subway"].apply(lambda x :findtransitscore(x))
df["transitscore"].value_counts()

5    38572
4     8345
2     1169
1      804
3      498
0      360
Name: transitscore, dtype: int64

In [11]:
df = df.drop(columns=["subway"])
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,transitscore
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.23,6,365,5
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,43,2019-01-02,0.38,2,365,5
2,3330,++ Brooklyn Penthouse Guestroom ++,4177,Jbee,Brooklyn,Williamsburg,40.70856,-73.94236,Private room,70,5,39,2018-12-07,0.35,3,217,5
3,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365,5
4,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,239,2019-03-03,4.43,1,164,5


In [13]:
df.to_csv('data/listings_with_transitscore.csv', header=True, index=None)